In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from collections import Counter
import os.path
from sklearn.metrics import pairwise_distances
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import pickle
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout
plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
HM_data = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
HM_data=HM_data.head(10000)

In [3]:
print("Number of data-points in the data:" , HM_data.shape[0])
print("Number of features in the data :" , HM_data.shape[1])

Number of data-points in the data: 10000
Number of features in the data : 25


In [4]:
print(HM_data.columns)

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')


In [5]:
images_path=[]
i=0
for p in HM_data['article_id'].tolist():
    path= '../input/h-and-m-personalized-fashion-recommendations/images/' + '0' + str(p)[:2] + '/'+ '0' + str(p) +'.jpg'

    if os.path.exists(path):
        i+=1
        images_path.append(path)
    else: images_path.append(None)
print(f'There is {i} article with corresponding image')
    

There is 9883 article with corresponding image


In [6]:
HM_data['image_path']=images_path
HM_data = HM_data[['article_id', 'prod_name', 'colour_group_name', 'image_path',
             'product_type_name', 'product_group_name', 'detail_desc']]
print("Number of data-points in the data:" , HM_data.shape[0])
print("Number of features in the data :" , HM_data.shape[1])

Number of data-points in the data: 10000
Number of features in the data : 7


In [7]:
HM_data.head(3)

,article_id,prod_name,colour_group_name,image_path,product_type_name,product_group_name,detail_desc
0,108775015,Strap top,Black,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
1,108775044,Strap top,White,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),Off White,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.


In [8]:
#finding the 10 most frequent product_type_names.
print(HM_data['product_type_name'].describe())

count        10000
unique          93
top       Trousers
freq          1080
Name: product_type_name, dtype: object


In [9]:
product_type_count = Counter(list(HM_data['product_type_name']))
product_type_count.most_common(10)

[('Trousers', 1080),
 ('Sweater', 789),
 ('T-shirt', 723),
 ('Dress', 500),
 ('Socks', 495),
 ('Underwear bottom', 449),
 ('Vest top', 434),
 ('Shorts', 368),
 ('Hoodie', 329),
 ('Shirt', 329)]

In [10]:
print(HM_data['colour_group_name'].describe())

count     10000
unique       49
top       Black
freq       2070
Name: colour_group_name, dtype: object


In [11]:
#find the 10 most frequent colors.
color_count = Counter(list(HM_data['colour_group_name']))
color_count.most_common(10)

[('Black', 2070),
 ('Dark Blue', 1361),
 ('White', 774),
 ('Light Pink', 553),
 ('Grey', 524),
 ('Dark Grey', 416),
 ('Dark Red', 338),
 ('Light Grey', 291),
 ('Red', 291),
 ('Blue', 284)]

In [12]:
print(HM_data['detail_desc'].describe())

count                                                  9944
unique                                                 3132
top       Sleeveless bodysuits in soft organic cotton je...
freq                                                     45
Name: detail_desc, dtype: object


In [13]:
print("Number of duplicates present in articles: {}".format(sum(HM_data.duplicated("detail_desc"))))

Number of duplicates present in articles: 6867


In [14]:
HM_data

,article_id,prod_name,colour_group_name,image_path,product_type_name,product_group_name,detail_desc
0,108775015,Strap top,Black,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
1,108775044,Strap top,White,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),Off White,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),Black,../input/h-and-m-personalized-fashion-recommen...,Bra,Underwear,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),White,../input/h-and-m-personalized-fashion-recommen...,Bra,Underwear,"Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...
9995,537708002,SEAN CREW NECK DEAL,Light Grey,../input/h-and-m-personalized-fashion-recommen...,Sweater,Garment Upper body,"Top in light, printed sweatshirt fabric with r..."
9996,537722001,BEAR KNOT TERRY,Grey,../input/h-and-m-personalized-fashion-recommen...,Sweater,Garment Upper body,Top in sweatshirt fabric with a decorative kno...
9997,537722004,BEAR KNOT TERRY,Pink,../input/h-and-m-personalized-fashion-recommen...,Sweater,Garment Upper body,Top in sweatshirt fabric with a decorative kno...
9998,537732001,Dita dress,Light Beige,None,Underdress,Underwear,"Microfibre slip with narrow, adjustable should..."


In [15]:

#removing data-points with short-titles
HM_data_sorted = HM_data[HM_data['detail_desc'].apply(lambda x: len(str(x).split())>4)]
print("After removal of products with short description:", HM_data_sorted.shape[0])

After removal of products with short description: 9851


In [16]:
HM_data_sorted.sort_values('detail_desc',inplace=True, ascending=False)

In [17]:
indices = []
for i,row in HM_data_sorted.iterrows():
    indices.append(i)
    
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = HM_data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
       
    previous_i = i
    a = HM_data['detail_desc'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:
        b = HM_data['detail_desc'].loc[indices[j]].split()        
        length = max(len(a), len(b))    
        count  = 0

        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(HM_data_sorted['article_id'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: stage1_dedupe_asins.append(HM_data_sorted['article_id'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break    

In [18]:
#removing duplicated
HM_data = HM_data.loc[HM_data['article_id'].isin(stage1_dedupe_asins)]

In [19]:
#getting the number of data-points after removing the duplicates
print('Number of data points : ', HM_data.shape[0])

Number of data points :  3062


In [20]:
indices = []
for i,row in HM_data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(HM_data['article_id'].loc[i])
    # consider the first apperal's title
    a = HM_data['detail_desc'].loc[i].split()
 
    for j in indices:
        
        b = HM_data['detail_desc'].loc[j].split()
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        #if the number of words in which both strings differ are < 3 ,
        #we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)

In [21]:
#getting the number of data-points after removing all the duplicates
print('Number of data points after stage two of dedupe: ',HM_data.shape[0])


Number of data points after stage two of dedupe:  3062


In [22]:
HM_data

,article_id,prod_name,colour_group_name,image_path,product_type_name,product_group_name,detail_desc
0,108775015,Strap top,Black,../input/h-and-m-personalized-fashion-recommen...,Vest top,Garment Upper body,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),Black,../input/h-and-m-personalized-fashion-recommen...,Bra,Underwear,"Microfibre T-shirt bra with underwired, moulde..."
7,111565003,20 den 1p Stockings,Beige,../input/h-and-m-personalized-fashion-recommen...,Socks,Socks & Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,Shape Up 30 den 1p Tights,Black,../input/h-and-m-personalized-fashion-recommen...,Leggings/Tights,Garment Lower body,Tights with built-in support to lift the botto...
9,111593001,Support 40 den 1p Tights,Black,../input/h-and-m-personalized-fashion-recommen...,Underwear Tights,Socks & Tights,"Semi shiny tights that shape the tummy, thighs..."
...,...,...,...,...,...,...,...
9985,537672001,Halloween Tattoos,Other Red,../input/h-and-m-personalized-fashion-recommen...,Fine cosmetics,Cosmetic,Temporary tattoos in different sizes.
9992,537688014,Rachel,Dark Grey,../input/h-and-m-personalized-fashion-recommen...,Sweater,Garment Upper body,Long polo-neck jumper in a soft knit with long...
9994,537691001,FRIDA furry jkt,Black,../input/h-and-m-personalized-fashion-recommen...,Jacket,Garment Upper body,"Jacket in soft faux fur with unlined sleeves, ..."
9997,537722004,BEAR KNOT TERRY,Pink,../input/h-and-m-personalized-fashion-recommen...,Sweater,Garment Upper body,Top in sweatshirt fabric with a decorative kno...


In [23]:
# removing stop-words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():    
            word = ("".join(e for e in words if e.isalnum()))      
            word = word.lower()       
            if not word in stop_words:
                string += word + " "
        HM_data[column][index] = string
        
for index, row in HM_data.iterrows():
    nlp_preprocessing(row['detail_desc'], index, 'detail_desc')

list of stop words: {'theirs', 'too', 'o', 'only', 'it', 'now', 'll', 'd', 'up', 'in', 'itself', "isn't", 'i', 'nor', 'very', 'herself', 'them', 'myself', 'a', 't', 'into', 'being', 'there', 'didn', 'out', 's', 'm', "you've", 'between', 'mustn', 'shouldn', 'wouldn', 'themselves', 'before', 'with', 'is', 'under', 'because', 'ain', 'that', 'weren', 'just', 'here', 'where', 'shan', 'few', 'its', 'wasn', 'who', 'about', 'your', 'own', 'what', 'did', 'mightn', 've', 'does', 'their', 'our', 'why', "you'll", 'by', "needn't", 'won', 'same', "wasn't", 'aren', 'for', 'am', 'no', 'has', "she's", "shouldn't", 'above', 'she', 'don', 'should', 'be', "weren't", 'which', 'down', "hasn't", 'most', 'do', 'we', 'and', 'than', 'needn', 'you', 'against', 'having', 'he', 'him', 'had', 'the', 'been', 'y', 'hasn', 'his', "shan't", 'are', 'so', 'these', 'after', 'to', 'her', 'doing', 'such', 'of', 'isn', "hadn't", 'from', 'those', "should've", 'this', "didn't", "wouldn't", 'couldn', 'other', 'hadn', 'yours', '

In [24]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
print(stemmer.stem('arguing'))
print(stemmer.stem('fishing'))


argu
fish


In [25]:
# function to display an image
def display_img(url,ax,fig):  
    img = Image.open(url) 
    plt.imshow(img)

In [26]:
# function to generate the heatmap
def plot_heatmap(keys, values, labels, url, text):    
        gs = gridspec.GridSpec(2, 2, width_ratios=[4,1], height_ratios=[4,1]) 
        fig = plt.figure(figsize=(25,3))
        # 1st, ploting heat map that represents the count of commonly ocurred words in title2
        ax = plt.subplot(gs[0])
        # it displays a cell in white color if the word is intersection(list of words of title1 and list of words of title2), in black if not
        ax = sns.heatmap(np.array([values]), annot=np.array([labels]))
        ax.set_xticklabels(keys) # set that axis labels as the words of title
        ax.set_title(text) # apparel title        
        # 2nd, plotting image of the the apparel
        ax = plt.subplot(gs[1])  
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])  
        display_img(url, ax, fig)           
        plt.show()

In [27]:
# function to display heatmap and image        
def plot_heatmap_image(doc_id, vec1, vec2, url, text, model):
      intersection = set(vec1.keys()) & set(vec2.keys())   
    for i in vec2:
        if i not in intersection:
            vec2[i]=0
   
    keys = list(vec2.keys())
    #  if ith word in intersection(lis of words of title1 and list of words of title2): values(i)=count of that word in title2 else values(i)=0 
    values = [vec2[x] for x in vec2.keys()]   
    if model == 'bag_of_words':
        labels = values
    elif model == 'tfidf':
        labels = []
        for x in vec2.keys(): 
            if x in  tfidf_title_vectorizer.vocabulary_:
                labels.append(tfidf_title_features[doc_id, tfidf_title_vectorizer.vocabulary_[x]])
            else:
                labels.append(0)
    elif model == 'idf':
        labels = []
        for x in vec2.keys():  
            if x in  idf_title_vectorizer.vocabulary_:
                labels.append(idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[x]])
            else:
                labels.append(0)
    plot_heatmap(keys, values, labels, url, text)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
#function to convert text into vector
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text) 
    return Counter(words) 

In [ ]:
# function to display the result
def get_result(doc_id, content_a, content_b, url, model):
    text1 = content_a
    text2 = content_b  
    vector1 = text_to_vector(text1) 
    vector2 = text_to_vector(text2)
    plot_heatmap_image(doc_id, vector1, vector2, url, text2, model)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(HM_data['detail_desc'])
title_features.get_shape()


In [ ]:
data=HM_data.copy()
def bag_of_words_model(doc_id, num_results): 
    pairwise_dist = pairwise_distances(title_features,title_features[doc_id])
  
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    #data frame indices of the 9 smallest distace's
    df_indices = list(data.index[indices])    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        get_result(indices[i],data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'bag_of_words')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('detail_desc:', data['detail_desc'].loc[df_indices[i]])
        print ('Euclidean similarity with the query image :', pdists[i])
        print('='*60)
#call the bag-of-words model for a product to get similar products
bag_of_words_model(1587, 20)
# in the output heat map each value represents the tfidf values of the label word, the color represents the intersection with inputs title

In [ ]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features = tfidf_title_vectorizer.fit_transform(data['detail_desc'])

In [ ]:
def tfidf_model(doc_id, num_results):   
    pairwise_dist = pairwise_distances(tfidf_title_features,tfidf_title_features[doc_id]) 
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]  
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]  
    df_indices = list(data.index[indices])
    for i in range(0,len(indices)):  
        get_result(indices[i], data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'tfidf')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('Eucliden distance from the given image :', pdists[i])
        print('='*125)
tfidf_model(1587, 20)


In [ ]:
idf_title_vectorizer = CountVectorizer()
idf_title_features = idf_title_vectorizer.fit_transform(data['detail_desc'])

In [ ]:
def n_containing(word): 
    return sum(1 for blob in data['detail_desc'] if word in blob.split())
def idf(word):
    return math.log(data.shape[0] / (n_containing(word)))

In [ ]:

idf_title_features  = idf_title_features.astype(np.float)
for i in idf_title_vectorizer.vocabulary_.keys():  
    idf_val = idf(i)
    # to calculate idf_title_features we will replace the count values with the idf values of the word
    for j in idf_title_features[:, idf_title_vectorizer.vocabulary_[i]].nonzero()[0]:        
        idf_title_features[j,idf_title_vectorizer.vocabulary_[i]] = idf_val  

In [ ]:
def idf_model(doc_id, num_results): 
    pairwise_dist = pairwise_distances(idf_title_features,idf_title_features[doc_id])
   
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
   
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
  
    df_indices = list(data.index[indices])
    for i in range(0,len(indices)):
        get_result(indices[i],data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'idf')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('euclidean distance from the given image :', pdists[i])
        print('='*125)        
idf_model(1587,20)
